# Обучение нейронности сети игре с помощью TensorFlow и OPEN AI

Целью проекта является демонстрация навыков в обучении с подкреплением. Целью не является создание модели высокого уровня, хотя при должном желании, проблема могла бы быть решена. 

Одна из больших проблем в обучении НС - это недостаток данных. В случае же с физикой мы можем создать уникальные данные, которые будут основаны на статистике и знании о явлении, поэтому мы можем искусственно сгенерировать всевозможный датасет для обучения НС. 

Еще один пример формирования сценария данных - это использование Open AI (Gym). 

gym.openai.com - сайт, которые включает в себя множество возможных сред для тренировки ИИ. Мы воспользуемся средой CartPole-v0

НС будет создавать сигналы, которые будут подаваться на игровую модель. В данном случае будет использоваться метод амсаблирования, когда несколько слабых моделей в совокупности будут давать более правильный (сильный) сигнал. 

In [28]:
import tensorflow as tf
import gym #формирует среду для тренировки модели 
import random #в игре вертикальная плашка движется рандомно. Использовав random мы сможем сгенерировать данные, похожие
#на движение плашки
import numpy as np
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 
from statistics import mean, median 
from collections import Counter 
import tflearn
tf.compat.v1.reset_default_graph()

In [29]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [30]:
#не дает модели использовать более 0.333 оперативки, помогает если надо обучать несколько моделей сразу (к примеру, когда
#мы учили несколько моделей сразу играть в игру)
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.99)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [31]:
LR = 1e-3 #learning_rate
env = gym.make('CartPole-v1') #окружение - игра, которой обучаем модель
env.reset()
goal_steps = 500 #Движение в игре происходит в кадрах. За каждый кадр, когда модель
#удерживает вертикальную линию мы получаем очки. goal_steps - количество кадров (перев. шагов до достижения цели). 
#сколько шагов мы даем модели для достижения цели
score_requirement = 50 #минимальное количество очков, которое должна набрать модель
initial_games = 10000 #количество игр, которые сыграет модель

In [32]:
#для начала будем создавать игры с рандомными данными:
def some_random_games_first():
    for episode in range(100): #episode - эпизод игры
        env.reset() #обновляем среду
        for t in range(goal_steps): # для каждого кадра из диапазона...
            env.render() #если хочешь видеть, что происходит в игре. Естественно, это замедлит обучение. Если не хочешь
            #видеть, то просто закомментируй
            action = env.action_space.sample() #действия, которые может совершать модель
            observation, reward, done, info = env.step(action)
            #observation - array из данных об игре. В данном случае просто позиция модели в каждом кадре.
            #reward - 1 или 0. Балансировала ли модель. Если да, то 1.
            #done - конец игры
            #info - другая информация
            if done:
                break 

Когда у нас есть окружение, которое мы можем генерировать таким путем, мы можем генерировать сэмплы для тренировки.

In [33]:
def initial_population(): #функция для создания данных
    training_data = [] #здесь будет observation и движения, которые были сделаны
    #все данные будут рандомные, но мы будем добавлять их в training_data только если очки за игру будут выше score_requirement
    scores = [] 
    accepted_scores = []
    
    for _ in range(initial_games): #для каждой игры, которую сыграет модель
        score = 0 #начальное количество очков
        game_memory = [] #до конца самой игры мы не будем знать свое количество очков, поэтому нам придется сохраняться
        #движения каждой модели 
        prev_observation = [] #лист с данными о прошлых играх 
        
        for _ in range(goal_steps): #для каждого кадра в игре. В игре всего будет goal_steps кадров 
            action = random.randrange(0,2) #генерирует только 0 и 1. Действия, которые совершает модель. Т.к. у модели только
            #два пути: двигаться влево или вправо. 
            observation, reward, done, info = env.step(action) #генерируем данные об шаге для каждой игры
            
            if len(prev_observation) > 0: #если в листе prev_observation уже есть наблюдения, то
                game_memory.append([prev_observation, action]) #позция модели в прошлом кадре и её действие
                #позиция в прошлом кадре, т.к. сначала obeservation зависит от env.step.action и сначала генерируется
                #действие потом, потом позиция модели в кадре
            
            #иначе, если в листе prev_observation нет наблюдений, туда попадает наблюдение, созданное первоначально
            prev_observation = observation
            
            score += reward #если модель сбалансировала в данном кадре, то прибавляется очко (которых нам надо 
            #набрать score_requirement)
            
            if done: #если все кадры (goal_steps) сыграны, то закончить
                break #заканчиваем цикл
                
        #теперь анализируем проведенную игру. Набрала ли модель нужно кол-во очков?
        #Для каждой сыгранной игры:
        if score >= score_requirement: #если набранное количество очков за игру больше требуемого
            accepted_scores.append(score) #мы принимаем очки и добавляем их в accepted_scores
            #если мы модель набрала больше, чем score_requirement очков, то записываем её действия
            for data in game_memory: #для данных об каждой игре
                #здесь создаются вектора движения. Т.к. у нас 2 действия, то конвертируем их в one-hot-enc
                #если модель action=1, то вектор [0, 1] и т.д. В общем это просто векторезированное действие.
                if data[1] == 1: #data[1] = action. Если action (котрое делали рандомно) = 1, то 
                    output = [0, 1] 
                elif data[1] == 0:
                    output = [1, 0]
                    
                training_data.append([data[0], output]) #добавляем в даные об каждой игре 
                #data[0] = prev_observation - данные о движение модели
                
        env.reset() #когда игра закончена обновляем окружение
        scores.append(score) #просто чтобы видеть все очки, которых модель достигла
        
    #игры закончились.
    training_data_save = np.array(training_data) #сохраняем данные и конвертируем их в array
    np.save('saved.np', training_data_save) 
    


    #отдел аналитик
    print('Average accepted score:', mean(accepted_scores))
    print('Median accepted score', median(accepted_scores))
    print(Counter(accepted_scores)) #показывает сколько раз модель достигла определенного очка
    #пр. 65.0 : 2, значит модель два раза достигла 65 очков
    
    return training_data 

С помощью tf можно сохранять модель, но чтобы загрузить эту сохраненную модель, нам нужно иметь модель, которая уже была инициализированна и на инпуты в нее должны заходить идентичные формы данных. Так что лучше разделять саму модель, её тренировку и её использование.

In [34]:
#инициализируем модель:
def neural_network_model(input_size):
    network = input_data(shape = [None, input_size, 1], name = 'input') #input_size в нашем случае 0, т.к. это observation.
    
    #создадим полносвязный лэйер
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8) #0.8 - это keep_rate, а не drop_rate. Drop_rate по дефолту также остается 0.2
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8) #0.8 - это keep_rate, а не drop_rate. Drop_rate по дефолту также остается 0.2
    
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8) #0.8 - это keep_rate, а не drop_rate. Drop_rate по дефолту также остается 0.2
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8) #0.8 - это keep_rate, а не drop_rate. Drop_rate по дефолту также остается 0.2
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8) #0.8 - это keep_rate, а не drop_rate. Drop_rate по дефолту также остается 0.2
    
    network = fully_connected(network, 2, activation='softmax') #2 - это количество опций модели куда двигаться
    #А что если в игре больше двух действий? Поговорим об этом ниже в markdown ячейке
    
    network = regression(network, optimizer = 'adam',
                         learning_rate=1e-3,
                         loss='categorical_crossentropy',
                         name = 'targets') 
    #задача регресси т.к. мы предсказываем на сколько 
    #должна двинуться модель, а не просто куда
    
    model = tflearn.DNN(network, tensorboard_dir='logs') 
    #можем сюда добавить подключение к tensorboard через укаазание tensorboard_dir = 'logs '
    #tensorboard создаст новую папку logs в месте, где хранится тетредь.

    return model

Мы делаем код максимально динамичным, чтобы он мог подстроиться к любой игре на gym. Допустим в игре 4 действия, тогда:

1) Изменяем саму игру env = gym.make('игра_name')


2) Изменяем score_requirement = ...


3) Смотрим сколько действия теперь есть для модели в action = env.action_space.sample()


4) Изменяем action = random.randrange(..., ...) на кол-во новых возможных действий 


5) network = fully_connected(network, ..., ) меняем кол-во аутпутов.

In [35]:
# функция для тренировки модели
def train_model(training_data, model=False):
    #если модель уже создана, то мы просто передадим её в функцию. Если модели еще нет, то фукнция создаст модель
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1)
    #-1 - мы "говорим" numpy о том, что хотим изменить форму array таким образом, чтобы он подходил под критерий 
    #len(training_data[0[0]])
    #Таким образом получаем np.array c формой 4 векторов, где есть 4 строки, 1 колонка, а векторы обернуты в 2 квадратных скоб.
    #training_data[0] - observation (движения модели в кадре)
    y = [i[1] for i in training_data]
    
    #если у нас еще нет модели, то
    if not model:
        model = neural_network_model(input_size=len(X[0]))
        
    #не стоит ставить много эпох, т.к. модель может переучиться. Если у нас на этапе обучения будет 95% accuracy, то скорее всего
    #у нас будет переобучение
    model.fit({'input':X}, {'targets':y}, n_epoch=4, snapshot_step=500, show_metric=True, run_id='openaistaff')
    
    return model

In [36]:
training_data = initial_population()

Average accepted score: 62.569060773480665
Median accepted score 58.0
Counter({52.0: 30, 50.0: 28, 51.0: 26, 56.0: 24, 53.0: 21, 55.0: 21, 60.0: 15, 57.0: 15, 61.0: 13, 54.0: 13, 62.0: 12, 64.0: 12, 58.0: 10, 68.0: 9, 65.0: 9, 59.0: 8, 73.0: 7, 66.0: 7, 63.0: 6, 69.0: 5, 84.0: 4, 75.0: 4, 74.0: 4, 78.0: 4, 90.0: 4, 71.0: 4, 80.0: 3, 92.0: 3, 67.0: 3, 86.0: 3, 87.0: 3, 77.0: 3, 72.0: 3, 82.0: 2, 85.0: 2, 79.0: 2, 120.0: 2, 70.0: 2, 103.0: 2, 88.0: 1, 89.0: 1, 117.0: 1, 109.0: 1, 128.0: 1, 115.0: 1, 100.0: 1, 81.0: 1, 101.0: 1, 95.0: 1, 76.0: 1, 110.0: 1, 104.0: 1, 83.0: 1})


C:\Users\yanka\AppData\Local\Temp\ipykernel_17752\3666262703.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data_save = np.array(training_data) #сохраняем данные и конвертируем их в array


In [37]:
model = train_model(training_data) #т.к. у нас еще нет модели 

Training Step: 1395  | total loss: 0.65966 | time: 4.344s
| Adam | epoch: 004 | loss: 0.65966 - acc: 0.6101 -- iter: 22272/22288
Training Step: 1396  | total loss: 0.66332 | time: 4.356s
| Adam | epoch: 004 | loss: 0.66332 - acc: 0.6100 -- iter: 22288/22288
--


In [38]:
#можем сохранить модель
#model.save('game_model.model')

#если захотим её загрузить позже, то можем просто сделать
#model.load(game_model.model)
#Таким образом нам не надо переучивать заново модель, запуская код. 

Accuracy в 61 процент тоже неплохо, но могло бы быть и лучше. Поработаем над этим.

In [39]:
scores = [] #очки за каждую игру
choices = [] #выбор модели куда двигаться

for each_game in range(10): #для каждой игры, которые будет range(N)
    score = 0
    game_memory = []
    prev_obs = []
    env.reset() #ресетим окружение перед началом каждой новой игры
    #для каждой игры, которая будет длиться goal_steps кадров
    for _ in range(goal_steps):
        env.render() #чтобы мы могли видеть игру
        
        #если нет никаких данных об среде в prev_obs, то создаем действие
        if len(prev_obs) == 0:
            action = random.randrange(0, 2)
        #но когда модель увидит первый кадр (куда движется палка)
        else:
            #action - должно быть 0 или 1. Аутпут из модели one-hot ([0, 1] или [1, 0])
            #через argmax берем индекс самого большого значения от предсказанного действия model.predict(prev_obs.)
            #модель видит первый кадр, и показывает действие, куда ей лучше двигаться. 
            #Х должен поступать на вход в определенной форме. Мы уже делали reshape для трейн данных и нужно сделать
            #такой же решейп для новых данных
            #берем 0 элемент из предикта модели, т.к. нам нужен ответ только на первый кадр
            #model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0] = array([0.64129007, 0.35870993], dtype=float32)
            #в предикте, т.к. мы используем loss=categorical_crossentropy, у нас хранится вероятность того, какой класс
            #больше подходит под инпут. 0.64129007 - вероятность принадлежать первом классу, а 0.35870993 - второму
            #таким образом мы с помощью argmax берем индекс наибольшей вероятности. 
            #[0] индекс в конце нужен, т.к. model.predict(prev_obs.reshape(-1, len(prev_obs), 1)) дает 
            # array([[0.64129007, 0.35870993]], dtype=float32). Проще говоря, мы просто избалвяемся от скобок [].
            #Можешь чекнкуть еще раз с помощью 
            #model.predict(np.array([training_data][0][0][0]).reshape(-1, len(training_data[0][0]), 1))[0]
            action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
            
        choices.append(action)
        
        new_observation, reward, done, info = env.step(action) #здесь данные о каждом шаге
        prev_obs = new_observation 
        #вообще нам не нужна линия ниже, если мы не собираемся доучивать модель.
        game_memory.append([new_observation, action]) #записываются данные о текущей игре. Состояние среды и действие модели
        #в ответ на среду
        score += reward #добавляем очки, которые модель получает за каждый кадр в игре
        
        if done:
            break
    
    #собираем очки за каждую игру
    scores.append(score)
    
print('Average score:', sum(scores)/len(scores))
#интересно посмотреть на сколько раз чаще модель отдает предпочтение первому действию перед вторым. Не происходит ли никакой
#ошибки?
print('Choice 1: {}, Choice 0: {}'.format(choices.count(1)/len(choices), choices.count(0)/len(choices)))           

Average score: 125.3
Choice 1: 0.5107741420590582, Choice 0: 0.48922585794094176
